# 双対法による制約付きMDPの解法

参考

* [Exploration-Exploitation in Constrained MDPs](https://arxiv.org/abs/2003.02189)
* [CONSTRAINED MARKOV DECISION PROCESSES](https://www-sop.inria.fr/members/Eitan.Altman/TEMP/h.pdf)（未読．読んでおこう．）

[前回](RL_CMDP_explore_exploit.ipynb)は線型計画法によってCMDPを解く方法を見ました．
ラグランジュの未定乗数法を使うと，動的計画法で解くこともできます．


表記

* 有限MDP: $\mathcal{M}=\left(\mathcal{S}, \mathcal{A}, c, p, s_1, H\right)$
    1. 有限状態集合: $S=\{1, \dots, |S|\}$
    2. 有限行動集合: $A=\{1, \dots, |A|\}$
    3. 非定常なコスト関数（元論文では確率変数ですが、ややこしいので決定的にします）: $c_h(s, a)$
    4. 非定常遷移確率: $p_h(s'|s, a)$
    5. 初期状態: $s_1$
    6. ホライゾン: $H$
    7. すべての状態行動の中で最大の次状態への遷移の数: $\mathcal{N}:=\max _{s, a, h}\left|\left\{s^{\prime}: p_h\left(s^{\prime} \mid s, a\right)>0\right\}\right|$
* 占有率：$q_h^\pi(s, a ; p):=\mathbb{E}\left[\mathbb{1}\left\{s_h=s, a_h=a\right\} \mid s_1=s_1, p, \pi\right]=\operatorname{Pr}\left\{s_h=s, a_h=a \mid s_1=s_1, p, \pi\right\}$
* 価値関数：$V_1^\pi\left(s_1 ; p, c\right)=\sum_{h, s, a} q_h^\pi(s, a ; p) c_h(s, a):=c^T q^\pi(p)$


* $\{d_i, \alpha_i\}_{i=1}^I$：$I$個の制約
    * $d_i \in \mathbb{R}^{SAH}$
    * $\alpha_i \in [0, H]$
    * $i$番目の制約（元論文では確率変数ですが、ややこしいので決定的にします）$d_{i, h}(s, a)$
    * $V_h^\pi\left(s ; p, d_i\right):=\mathbb{E}\left[\sum_{h^{\prime}=h}^H d_{i, h^{\prime}}\left(s_{h^{\prime}}, a_{h^{\prime}}\right) \mid s_h=s, p, \pi\right]$.

CMDPについて振り返ってみましょう．CMDPの目的は次の最適方策の導出です．

$$
\begin{gathered}
\pi^{\star} \in \underset{\pi \in \Pi^{\mathrm{MR}}}{\arg \min } c^T q^\pi(p) \\
\text { s.t. } D q^\pi(p) \leq \alpha
\end{gathered}
$$

ここで、
$$
D=\left[\begin{array}{c}
d_1^T \\
\vdots \\
d_I^T
\end{array}\right], \quad \alpha=\left[\begin{array}{c}
\alpha_1 \\
\vdots \\
\alpha_I
\end{array}\right]
$$

としました．
これはラグランジュ未定乗数$\lambda \in \mathbb{R}_{+}^I$を使うことで，次の式に直すことができます：

$$
L^*=\max _{\lambda \in \mathbb{R}_{+}^I} \min _{\pi \in \Delta_A^S}\left\{c^T q^\pi(p)+\lambda^T\left(D q^\pi(p)-\alpha\right)\right\}
$$

このCMDPには強双対性が成り立つので，このミニマックスゲームは$L^*-V_1^*(s_1)$を満たします．
これを解くことを考えます．


## 動的計画法による解法

まず次の問題を解きます．

$$
\pi_k \in \underset{\pi \in \Pi^{\mathrm{MR}}}{\arg \min }\left({c}+{D}^T \lambda_k\right)^{\top} q^\pi\left(p\right)-\lambda_k^T \alpha
$$

これはMDP $\mathcal{M}_k = \left\{M=(S, A, r^+, p): r_h^{+}(s, a)={c}_h(s, a)+\sum_i\left(d_{i, h}(s, a)-\alpha_i\right) \lambda_i^k\right\}$の最適方策を求めれば良いので，次の価値反復法で解けます：

$$
Q_h^k(s, a)=r_h^{+}(s, a)+ \sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \min _{a^{\prime}} Q_{h+1}^k\left(s^{\prime}, a^{\prime}\right)
$$

続いてラグランジュ未定乗数を更新します．

$$
\lambda_{k+1}=\left[\lambda_k+\frac{1}{t_\lambda}\left({D} q^{\pi_k}\left({p}\right)-\alpha\right)\right]
$$

ここで出てくる占有率$q^{\pi_k}$も簡単な反復法で計算できます．（$t_\lambda$は少し厄介ですが，なんらかの小さい定数で置いておきます）

この２つを繰り返すと，CMDPの最適方策に収束します．確認してみましょう．

In [31]:
import jax
import jax.numpy as jnp
import numpy as np
from typing import NamedTuple, Optional
from jax.random import PRNGKey

key = PRNGKey(0)

S = 7  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
H = 5  # ホライゾン

# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(H, S, A))
assert rew.shape == (H, S, A)


# コスト行列を適当に作ります
key, _ = jax.random.split(key)
cost = jax.random.uniform(key=key, shape=(H, S, A))
assert cost.shape == (H, S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(H, S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(H, S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
init_dist = jax.random.uniform(key, shape=(S,))
init_dist = init_dist / jnp.sum(init_dist)
np.testing.assert_allclose(init_dist.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class CMDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    H: int  # ホライゾン
    rew: jnp.array  # 報酬行列
    cost: jnp.array  # 報酬行列
    const: float  # 制約の閾値
    P: jnp.array  # 遷移確率行列
    init_dist: jnp.array  # 初期分布
    optimal_V_rew: Optional[jnp.ndarray] = None  # 報酬についての最適V値
    optimal_V_cost: Optional[jnp.ndarray] = None  # コストについての最適V値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


const = 0.3 * H  # 制約は適当です。このときに実行可能である保証はとくにありません。
mdp = CMDP(S_set, A_set, H, rew, cost, const, P, init_dist)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("ホライゾン：", mdp.H)
print("制約：", mdp.const)

状態数： 7
行動数： 3
ホライゾン： 5
制約： 1.5


In [32]:
import jax
import chex


@jax.jit
def compute_policy_Q(mdp: CMDP, policy: jnp.ndarray):
    """ベルマン期待作用素をホライゾン回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (CMDP)
        policy (np.ndarray): (HxSxA)の行列

    Returns:
        policy_Q_rew (jnp.ndarray): (HxSxA)の行列. 報酬関数についてのQ
        policy_Q_cost (jnp.ndarray): (HxSxA)の行列. コスト関数についてのQ
    """
    H, S, A = policy.shape

    def backup(i, args):
        policy_Q, g = args
        h = H - i - 1
        max_Q = (policy[h+1] * policy_Q[h+1]).sum(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        policy_Q = policy_Q.at[h].set(g[h] + next_v)
        return policy_Q, g
    
    policy_Q_rew = jnp.zeros((H+1, S, A))
    args = policy_Q_rew, mdp.rew
    policy_Q_rew, _ = jax.lax.fori_loop(0, mdp.H, backup, args)

    policy_Q_cost = jnp.zeros((H+1, S, A))
    args = policy_Q_cost, mdp.cost
    policy_Q_cost, _ = jax.lax.fori_loop(0, mdp.H, backup, args)
    return policy_Q_rew[:-1], policy_Q_cost[:-1]


まず線型計画法で解いてみます．

In [37]:
from scipy.optimize import linprog
from itertools import product
B = np.zeros((H, S, A, H, S, A))

# 初期状態についての制約
for s, a in product(range(S), range(A)):
    B[0, s, a, 0, s] = 1


# 遷移についての制約
for h, s, a in product(range(1, H), range(S), range(A)):
    B[h, s, a, h, s] = 1  # sum_a d(h, s, a) を実現します
    B[h, s, a, h-1] = -mdp.P[h-1, :, :, s]  # sum_a d(h, s, a) を実現します


B = B.reshape((H*S*A, H*S*A))
mu = np.repeat(mdp.init_dist[:, None], A, axis=1).reshape(-1)
b = np.hstack((mu, np.zeros((H-1)*S*A)))

# コストについての制約
C = mdp.cost.reshape(1, -1)
c = np.array((mdp.const,))

r = - mdp.rew.reshape(-1)
lin_res = linprog(r, A_eq=B, b_eq=b, bounds=(0, None), A_ub=C, b_ub=c)

d_arr = lin_res.x.reshape(H, S, A)
np.testing.assert_allclose(d_arr.sum(axis=(1, 2)), 1.0, atol=1e-4)

# 行動の確率が全て０の箇所はUniformにします（この状態には訪れないことを意味しますが、NanがでちゃうのでUniformで回避します）
optimal_policy = d_arr / d_arr.sum(axis=-1, keepdims=True)
optimal_policy = jnp.where(jnp.isnan(optimal_policy), 1 / mdp.A, optimal_policy)
Q_rew, Q_cost = compute_policy_Q(mdp, optimal_policy)
V_rew, V_cost = (Q_rew * optimal_policy).sum(axis=-1), (Q_cost * optimal_policy).sum(axis=-1)

total_cost = V_cost[0] @ mdp.init_dist
assert total_cost <= mdp.const
print("最適方策の累積コスト和", total_cost)

total_rew = V_rew[0] @ mdp.init_dist
print("最適方策の累積報酬和", total_rew)

mdp = mdp._replace(optimal_V_rew=V_rew, optimal_V_cost=V_cost)

最適方策の累積コスト和 1.5
最適方策の累積報酬和 3.2056034


続いて双対問題を解きます．

In [39]:
from functools import partial
import jax
import chex


@jax.jit
def compute_greedy_policy(Q: jnp.ndarray):
    """Q関数の貪欲方策を返します

    Args:
        Q (jnp.ndarray): (HxSxA)の行列

    Returns:
        greedy_policy (jnp.ndarray): (HxSxA)の行列
    """
    greedy_policy = jnp.zeros_like(Q)
    H, S, A = Q.shape
    
    def body_fn(i, greedy_policy):
        greedy_policy = greedy_policy.at[i, jnp.arange(S), Q[i].argmax(axis=-1)].set(1)
        return greedy_policy

    greedy_policy = jax.lax.fori_loop(0, H, body_fn, greedy_policy)
    chex.assert_shape(greedy_policy, (H, S, A))
    return greedy_policy


@partial(jax.jit, static_argnames=("H", "S", "A"))
def _compute_optimal_Q(mdp: CMDP, H: int, S: int, A: int):
    """ベルマン最適作用素をホライゾン回走らせて最適価値関数を動的計画法で計算します。
    Args:
        mdp (CMDP)

    Returns:
        optimal_Q (jnp.ndarray): (HxSxA)の行列
    """

    def backup(i, optimal_Q):
        h = H - i - 1
        max_Q = optimal_Q[h+1].max(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        optimal_Q = optimal_Q.at[h].set(mdp.rew[h] + next_v)
        return optimal_Q
    
    optimal_Q = jnp.zeros((H+1, S, A))
    optimal_Q = jax.lax.fori_loop(0, mdp.H, backup, optimal_Q)
    return optimal_Q[:-1]

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp, mdp.H, mdp.S, mdp.A)


@jax.jit
def compute_policy_Q(mdp: CMDP, policy: jnp.ndarray):
    """ベルマン期待作用素をホライゾン回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (CMDP)
        policy (np.ndarray): (HxSxA)の行列

    Returns:
        policy_Q_rew (jnp.ndarray): (HxSxA)の行列. 報酬関数についてのQ
        policy_Q_cost (jnp.ndarray): (HxSxA)の行列. コスト関数についてのQ
    """
    H, S, A = policy.shape

    def backup(i, args):
        policy_Q, g = args
        h = H - i - 1
        max_Q = (policy[h+1] * policy_Q[h+1]).sum(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        policy_Q = policy_Q.at[h].set(g[h] + next_v)
        return policy_Q, g
    
    policy_Q_rew = jnp.zeros((H+1, S, A))
    args = policy_Q_rew, mdp.rew
    policy_Q_rew, _ = jax.lax.fori_loop(0, mdp.H, backup, args)

    policy_Q_cost = jnp.zeros((H+1, S, A))
    args = policy_Q_cost, mdp.cost
    policy_Q_cost, _ = jax.lax.fori_loop(0, mdp.H, backup, args)
    return policy_Q_rew[:-1], policy_Q_cost[:-1]



@jax.jit
def compute_policy_matrix(policy: jnp.ndarray):
    """
    上で定義した方策行列を計算します。方策についての内積が取りたいときに便利です。
    Args:
        policy (jnp.ndarray): (HxSxA)の行列

    Returns:
        policy_matrix (jnp.ndarray): (HxSxSA)の行列
    """
    H, S, A = policy.shape
    PI = policy.reshape(H, 1, S, A)
    PI = jnp.tile(PI, (1, S, 1, 1))
    eyes = jnp.tile(jnp.eye(S).reshape(1, S, S, 1), (H, 1, 1, 1))
    PI = (eyes * PI).reshape(H, S, S*A)
    return PI


@jax.jit
def compute_policy_visit(mdp: CMDP, policy: jnp.ndarray, init_dist: jnp.ndarray):
    """MDPと方策について、訪問頻度を動的計画法で計算します。
    Args:
        mdp (CMDP)
        policy (jnp.ndarray): (HxSxA)の行列
        init_dist (jnp.ndarray): (S) 初期状態の分布

    Returns:
        visit (jnp.ndarray): (HxSxA)のベクトル
    """
    H, S, A = policy.shape
    Pi = compute_policy_matrix(policy)
    P = mdp.P.reshape(H, S*A, S)

    def body_fn(h, visit):
        next_visit = visit[h] @ P[h] @ Pi[h+1]
        visit = visit.at[h+1].set(next_visit)
        return visit
    
    visit = jnp.zeros((H+1, S*A))
    visit = visit.at[0].set((init_dist @ Pi[0]))
    visit = jax.lax.fori_loop(0, mdp.H, body_fn, visit)
    visit = visit[:-1].reshape(H, S, A)
    return visit


In [40]:
@partial(jax.jit, static_argnames=("H", "S", "A"))
def _solve_dual_CMDP(mdp: CMDP, H: int, S: int, A: int, num_iter: int=1000, lam_coef: float=0.01):
    """双対問題を通じてCMDPを解きます．
    Args:
        mdp (CMDP)

    Returns:
        optimal_policy (jnp.ndarray): (HxSxA)の行列
    """

    def loop_fn(k, lam):
        reg_rew = mdp.rew - (mdp.cost - mdp.const) * lam
        reg_mdp = mdp._replace(rew=reg_rew)
        Q = _compute_optimal_Q(reg_mdp, H, S, A)
        new_policy = compute_greedy_policy(Q)
        new_policy_visit = compute_policy_visit(reg_mdp, new_policy, reg_mdp.init_dist)

        new_lam = lam + lam_coef * (reg_mdp.cost.reshape(-1) @ new_policy_visit.reshape(-1) - mdp.const)
        return new_lam
    
    lam = 1.0
    lam = jax.lax.fori_loop(0, num_iter, loop_fn, lam)

    reg_rew = mdp.rew - (mdp.cost - mdp.const) * lam
    reg_mdp = mdp._replace(rew=reg_rew)
    Q = _compute_optimal_Q(reg_mdp, H, S, A)
    policy = compute_greedy_policy(Q)

    return policy


solve_dual_CMDP = lambda mdp, num_iter, lam_coef: _solve_dual_CMDP(mdp, mdp.H, mdp.S, mdp.A, num_iter, lam_coef)
dual_policy = solve_dual_CMDP(mdp, 10000, 0.001)

Q_rew, Q_cost = compute_policy_Q(mdp, dual_policy)
V_rew, V_cost = (Q_rew * dual_policy).sum(axis=-1), (Q_cost * dual_policy).sum(axis=-1)

dual_total_cost = V_cost[0] @ mdp.init_dist
print("最適方策の累積コスト和", dual_total_cost)

dual_total_rew = V_rew[0] @ mdp.init_dist
print("最適方策の累積報酬和", dual_total_rew)

最適方策の累積コスト和 1.5063109
最適方策の累積報酬和 3.213574


In [41]:
print("線形計画による累積コスト和", total_cost)
print("双対法による累積コスト和", dual_total_cost)

print("線型計画法による累積報酬和", total_rew)
print("双対法による累積報酬和", dual_total_rew)

線形計画による累積コスト和 1.5
双対法による累積コスト和 1.5063109
線型計画法による累積報酬和 3.2056034
双対法による累積報酬和 3.213574


双対法で近い解が得られていますが，ちょっと制約をオーバーしていますね．
これが数値的な問題なのか，それとも仕組み的なものなのかは微妙です．
実際，線型計画法で求めた方策は確率的方策ですが，双対法で求めたのは決定的方策になっています．
（TODO: ）